Vamos a proceder a hacer una serie de pruebas de generación automática de preguntas. Voy a seguir los ejemplos que se encuentran en https://github.com/asahi417/lm-question-generation?tab=readme-ov-file

Primero cargamos el excel con el corpus

In [ ]:
pip install openpyxl

In [ ]:
pip install spacy

In [ ]:
!python3 -m spacy download es_core_news_sm

In [ ]:
import pandas as pd
from pprint import pprint

In [ ]:
corpus = pd.read_excel('corpus_inicial.xlsx')

In [ ]:
corpus = corpus.rename(columns={"Column1": "original", "Column2": "gpt","Column3":"manual"})

In [ ]:
corpus = corpus.drop(0)

In [ ]:
corpus = corpus.reset_index(drop=True)

In [ ]:
corpus

,original,gpt,manual,Column4
0,4.º Que el Almanak ha de estar impreso y de ve...,El Almanak debe ser impreso y estar a la venta...,4.º El calendario debe ser impreso y estar a l...,¿Qué día se debe imprimir el calendario?
1,En este discurso dedicado al Supremo Consejo y...,En este discurso dedicado al Supremo Consejo y...,En este discurso dedicado al Supremo Consejo y...,¿Como son generalmente la Leyes formularias?
2,De 12½ á una del dia 16 del pasado se perdió u...,"El día 16 del mes pasado, a las 12:30 de la ta...","El día 16 del mes pasado, entre las 12:30 y la...",¿Qué se perdió?
3,5.º Que si para el dia 1.º de noviembre no est...,Si el Almanak no está a la venta para el 1 de ...,5.º Si el Calendario no está a la venta para e...,¿Qué pasa si el calendario no está a la venta ...
4,¡Quién le ha dicho á ese maligno que todos los...,¿Quién le ha dicho a esa persona malintenciona...,¡Quién le ha dicho a esa persona malintenciona...,¿Está haciendo una comparación?
...,...,...,...,...
605,Se necesita para una de las bóticas mas antigu...,"Se busca un regente o gerente, con experiencia...","Se busca un gerente, con experiencia y buen ju...",¿Que edad se requiere para el trabajo?
606,"á sus razones, tal fuerza á sus reparos, tal p...","El autor tiene argumentos, objeciones y prueba...","Tiene argumentos, objeciones y pruebas que pue...",¿Que se intenta hacer al aportar los argumento...
607,se sirva insertar este aviso en su periódico p...,"Por favor, publique este aviso en su periódico...","Por favor, publique este aviso en su periódico...",¿A que horas hay que presentarse?
608,Señor editor del diario: mur señor o El singul...,Señor editor del periódico: Me complace inform...,Señor editor del periódico: Me complace inform...,¿A quien va dirigido el escrito?


In [ ]:
corpus['iguales'] = corpus.apply(lambda x: x['gpt']==x['manual'], axis=1)

In [ ]:
corpus.head()

,original,gpt,manual,iguales
0,4.º Que el Almanak ha de estar impreso y de ve...,El Almanak debe ser impreso y estar a la venta...,4.º El calendario debe ser impreso y estar a l...,False
1,En este discurso dedicado al Supremo Consejo y...,En este discurso dedicado al Supremo Consejo y...,En este discurso dedicado al Supremo Consejo y...,False
2,De 12½ á una del dia 16 del pasado se perdió u...,"El día 16 del mes pasado, a las 12:30 de la ta...","El día 16 del mes pasado, entre las 12:30 y la...",False
3,5.º Que si para el dia 1.º de noviembre no est...,Si el Almanak no está a la venta para el 1 de ...,5.º Si el Calendario no está a la venta para e...,False
4,¡Quién le ha dicho á ese maligno que todos los...,¿Quién le ha dicho a esa persona malintenciona...,¡Quién le ha dicho a esa persona malintenciona...,False


In [ ]:
frases_distintas = len(corpus.loc[corpus['iguales']== False])
frases_iguales = len(corpus.loc[corpus['iguales']== True])

print('Tenemos %s frases' % (len(corpus)))
print('Tenemos %s frases iguales, un %s porciento' % (frases_iguales, round((frases_iguales*100)/len(corpus),2) ))
print('Tenemos %s frases distintas, un %s porciento' % (frases_distintas, round((frases_distintas*100)/len(corpus),2)))

Tenemos 627 frases
Tenemos 246 frases iguales, un 39.23 porciento
Tenemos 381 frases distintas, un 60.77 porciento


Esto quiere decir que el 61 % de las frases que ha propuesto el modelo de gpt han sido modificadas. Para la generación sintética de preguntas puede ser interesante ver que tipo de preguntas se generan en cada uno de los casos. Nos centramos ahora en eso.


# Generación de preguntas en cada uno de los 3 casos.

Cargamos especificando la lengua de nuestros textos. Sin embargo los desarrolladores tienen varios modelos creados y podría ser interesante

In [ ]:
model = TransformersQG(language="es")

/usr/local/lib/python3.10/dist-packages/transformers/models/auto/tokenization_auto.py:757: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/models/auto/configuration_auto.py:913: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/transformers/modeling_utils.py:2991: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
/usr/local/lib/

Hacemos pruebas para un solo caso y vamos viendo

In [ ]:
corpus['original'][1]

'4.º Que el Almanak ha de estar impreso y de venta al público en 1.º de noviembre del presente año sin falta alguna; y ha de venderse al precio de un real de vellon cada egemplar impreso de dos pliegos, y al de siete cuartos el que lo esté en uno, y no mas.'

In [ ]:
context = [corpus['original'][1],corpus['original'][1]]
respuesta = ['1.º de noviembre','un real de vellon']

# model prediction
question = model.generate_q(list_context=context, list_answer=respuesta)
pprint(question)


100%|██████████| 2/2 [00:00<00:00, 516.19it/s]


['¿En qué fecha se imprime el Almanak a público?',
 '¿Cuál es el precio de venderse al público en noviembre de noviembre del '
 'presente año?']


In [ ]:
corpus['gpt'][1]

'El Almanak debe ser impreso y estar a la venta al público el 1 de noviembre de este año sin excepción. Debe venderse al precio de un real de vellón cada ejemplar impreso de dos pliegos, y al precio de siete cuartos el que esté en uno, y no más.'

In [ ]:
context = [corpus['gpt'][1],corpus['gpt'][1]]
respuesta = ['1 de noviembre','un real de vellón']

# model prediction
question = model.generate_q(list_context=context, list_answer=respuesta)
pprint(question)


100%|██████████| 2/2 [00:00<00:00, 773.50it/s]


['¿En qué fecha se puede vender el Almanak al público?',
 '¿Qué debe venderse el Almanak?']


In [ ]:
corpus['manual'][1]

'4.º El calendario debe ser impreso y estar a la venta al público el 1 de noviembre de este año sin excepción. Debe venderse al precio de un real de moneda cada ejemplar impreso de dos pliegos, y al precio de siete cuartos el que esté en uno, y no más.'

In [ ]:
context = [corpus['manual'][1],corpus['manual'][1]]
respuesta = ['1 de noviembre','un real de moneda']

# model prediction
question = model.generate_q(list_context=context, list_answer=respuesta)
pprint(question)

100%|██████████| 2/2 [00:00<00:00, 493.91it/s]


['¿En qué fecha se puede vender el calendario a la venta al público?',
 '¿Qué debe venderse cada ejemplar impreso de dos pliegos?']


En estos casos le proporcionamos el contexto y la respuesta que queremos y nos genera una pregunta. Podemos ver como las preguntas tienen sentido.


# Solo con el contexto

In [ ]:
# initialize model
model = TransformersQG(language='es')

In [ ]:
context = corpus['manual'][1]
# model prediction
question_answer = model.generate_qa(context)

100%|██████████| 2/2 [00:00<00:00, 587.93it/s]


In [ ]:
pprint(question_answer)

[('¿Qué debe ser el calendario?',
  'impreso y estar a la venta al público el 1 de noviembre de este año'),
 ('¿Dónde debe venderse el calendario?',
  'al precio de un real de moneda cada ejemplar impreso de dos pliegos, y al '
  'precio de siete cuartos el que esté en uno, y no')]


Los pares pregunta-respuesta dejan un poco que desear, los desarrolladores se han centrado en textos en inglés.

In [ ]:
corpus['manual'][2]

'En este discurso dedicado al Supremo Consejo y a cuya publicación precedió la vista de los tres Señores Fiscales, se habla de la importancia de las Leyes formularias, que son acusadas generalmente de ser vacías y supersticiosas. Se mencionan los daños que causan las leyes bien dictadas, pero que no tienen fórmulas definidas para los actos que se tratan. También se demuestran los riesgos de contradicción y los innumerables beneficios que se obtendrán aplicando el formulario de los juicios, evitando la prolongación de los juicios y reduciendo el número de abogados y otras personas que trabajan en el Foro.'

In [ ]:
context = corpus['manual'][2]
# model prediction
question_answer = model.generate_qa(context)

100%|██████████| 3/3 [00:00<00:00, 659.83it/s]


In [ ]:
pprint(question_answer)

[('¿A quién se dedica el discurso dedicado a los tres Señores Fiscales?',
  'Supremo Consejo'),
 ('¿Cuáles son los riesgos que causan las leyes bien dictadas, pero no tienen '
  'fórmulas definidas para los actos que se tratan?',
  'los daños que causan las leyes bien dictadas, pero que no tienen fórmulas '
  'definidas para los actos que se tratan.'),
 ('¿Cuáles son los riesgos de contradicción y los innumerables beneficios que '
  'se obtendrán aplicando el formulario de los juicios?',
  'riesgos de contradicción y los innumerables beneficios que se obtendrán '
  'aplicando el formulario de los juicios, evitando la prolongación de los '
  'juicios y reduciendo el número de abogados y otras personas que trabajan en '
  'el Foro.')]


Como podemos ver estas preguntas-respuestas no son válidad ya que ni están bien formuladas ni bien respondidas.

# Prompt LLAMA_index GPT3.5 Turbo

Estoy teniendo problemas con el tema del Precio para usar openAI y no me entero

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
%pip install llama-index-llms-openai
%pip install llama-index
%pip install llama-index-llms-openai-like


In [ ]:
import openai
import os

openai.api_key="your-token"

print(openai.api_key)

In [ ]:
import logging
import sys
import pandas as pd

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
from llama_index.core.evaluation import DatasetGenerator, RelevancyEvaluator
from llama_index.core.llama_dataset.generator import RagDatasetGenerator
from llama_index.core import SimpleDirectoryReader, VectorStoreIndex, Response
from llama_index.llms.openai import OpenAI
import nest_asyncio
from llama_index.llms.openai_like import OpenAILike

In [ ]:
#Primeras 100 preguntas
reader = SimpleDirectoryReader("Original0-100")
documents = reader.load_data()

In [ ]:
#Segunda vuelta
llm = OpenAI(model="gpt-4o-mini", temperature=0.7)

In [ ]:
data_generator = RagDatasetGenerator.from_documents(documents,llm=llm)


In [ ]:
async def main():
    print(1)

await main()

1


In [ ]:
nest_asyncio.apply()

In [ ]:
eval_questions = data_generator.generate_questions_from_nodes()

In [ ]:
pandas_df = eval_questions.to_pandas()

In [ ]:
pandas_df.head(10)

,query,reference_contexts,reference_answer,reference_answer_by,query_by
0,"Según el texto, ¿cuál es la fecha límite para ...",[4.º Que el Almanak ha de estar impreso y de v...,,None,ai (gpt-4o-mini)
1,¿Cuál es el precio establecido para cada ejemp...,[4.º Que el Almanak ha de estar impreso y de v...,,None,ai (gpt-4o-mini)
2,¿Qué precio se debe pagar por un ejemplar impr...,[4.º Que el Almanak ha de estar impreso y de v...,,None,ai (gpt-4o-mini)
3,¿Cuál es el enfoque principal del discurso men...,[En este discurso dedicado al Supremo Consejo ...,,None,ai (gpt-4o-mini)
4,"Según el discurso, ¿qué daños se mencionan com...",[En este discurso dedicado al Supremo Consejo ...,,None,ai (gpt-4o-mini)
5,¿Qué beneficios se destacan en el texto como r...,[En este discurso dedicado al Supremo Consejo ...,,None,ai (gpt-4o-mini)
6,¿Qué evento ocurrió el día 9 del pasado en el ...,[Quien hubiere perdido una yegua que se halló ...,,None,ai (gpt-4o-mini)
7,¿A quién debe acudir una persona que haya perd...,[Quien hubiere perdido una yegua que se halló ...,,None,ai (gpt-4o-mini)
8,¿Quién es el Sr. Alcalde de la Hermandad menci...,[Quien hubiere perdido una yegua que se halló ...,,None,ai (gpt-4o-mini)
9,¿A qué hora se extravió la burra de leche segú...,[Entre 7 y 8 de la mañana del 29 del pasado se...,,None,ai (gpt-4o-mini)


In [ ]:
pandas_df.to_csv('questions_original0-100_gpt4-mini.csv')

In [ ]:
eval_questions.save_json("rag_dataset500-600.json")

In [ ]:
pandas_df['query'][0]

'What is being celebrated at the Convento de Religiosas Capuchinas on the fifth day of prayer?'

In [ ]:
pandas_df.to_csv('questions_manual0-100_gpt4.csv')

# Mistral


In [ ]:
%pip install llama-index
%pip install transformers accelerate bitsandbytes
%pip install llama-index-readers-web
%pip install llama-index-llms-huggingface
%pip install llama-index-embeddings-huggingface
%pip install llama-index-program-openai
%pip install llama-index-agent-openai

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 41.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.5/1.5 MB 59.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 51.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.2/180.2 kB 14.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 5.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 365.7/365.7 kB 18.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.8/295.8 kB 20.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 56.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 48.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 239.5/239.5 kB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 616.0/616.0 kB 

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
import torch
from transformers import BitsAndBytesConfig
from llama_index.core.prompts import PromptTemplate
from llama_index.llms.huggingface import HuggingFaceLLM

quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)


llm = HuggingFaceLLM(
    model_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
    tokenizer_name="mistralai/Mixtral-8x7B-Instruct-v0.1",
    query_wrapper_prompt=PromptTemplate("<s>[INST] {query_str} [/INST] </s>\n"),
    context_window=3900,
    max_new_tokens=256,
    model_kwargs={"quantization_config": quantization_config},
    # tokenizer_kwargs={},
    generate_kwargs={"temperature": 0.2, "top_k": 5, "top_p": 0.95},
    device_map="auto",
)

/usr/local/lib/python3.10/dist-packages/pydantic/_internal/_fields.py:161: UserWarning: Field "model_id" has conflict with protected namespace "model_".

You may be able to resolve this warning by setting `model_config['protected_namespaces'] = ()`.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json:   0%|          | 0.00/720 [00:00<?, ?B/s]

RuntimeError: No GPU found. A GPU is needed for quantization.

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer

model_id = "mistralai/Mixtral-8x7B-Instruct-v0.1"
tokenizer = AutoTokenizer.from_pretrained(model_id)

model = AutoModelForCausalLM.from_pretrained(model_id, device_map="auto")

model-00019-of-00019.safetensors:   0%|          | 0.00/4.22G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/19 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

In [ ]:
 from llama_index.core import SimpleDirectoryReader

In [ ]:
reader = SimpleDirectoryReader("Manual0-100")
documents = reader.load_data()

In [ ]:
from llama_index.core import VectorStoreIndex

vector_index = VectorStoreIndex.from_documents(documents)

In [ ]:
from llama_index.core.indices import SummaryIndex

summary_index = SummaryIndex.from_documents(documents)

In [ ]:
from llama_index.core.response.notebook_utils import display_response

In [ ]:
import logging
import sys

logging.basicConfig(stream=sys.stdout, level=logging.INFO)
logging.getLogger().addHandler(logging.StreamHandler(stream=sys.stdout))

In [ ]:
mistral = pd.read_csv('preguntas_mistral_original_manual.csv')

In [ ]:
mistral = mistral.rename(columns={'manual':'frase','Preguntas_manual':'pregunta_generada'})[[ "frase","pregunta_generada"]]
mistral["pregunta_generada"] = mistral["pregunta_generada"].apply(lambda x: str(x).replace('? ¿','? \n ¿') )

In [ ]:
# Dividir la columna en filas manteniendo las otras columnas
corpus_mixtral_expanded = mistral.set_index(['frase'])['pregunta_generada'].str.split('\n').apply(pd.Series).stack().reset_index()

# Renombrar las columnas resultantes
corpus_mixtral_expanded.columns = ['frase','level', 'pregunta_generada']
corpus_mixtral_expanded["pregunta_generada"] = corpus_mixtral_expanded["pregunta_generada"].apply(lambda x: str(x).replace('1. ','').replace('2. ','').replace('3. ','') )

In [ ]:
corpus_mixtral_expanded = corpus_mixtral_expanded.drop('level',axis=1)

In [ ]:
corpus_mixtral_expanded.head()

,frase,pregunta_generada
0,4.º El calendario debe ser impreso y estar a l...,¿Cuál es la fecha límite para que el Almanak e...
1,4.º El calendario debe ser impreso y estar a l...,¿A qué precio se debe vender cada ejemplar imp...
2,4.º El calendario debe ser impreso y estar a l...,¿Cuál será la consecuencia si el Almanak no es...
3,En este discurso dedicado al Supremo Consejo y...,¿Qué se discute en el discurso dirigido al Sup...
4,En este discurso dedicado al Supremo Consejo y...,¿Cuáles son los daños que se mencionan en el t...


In [ ]:
def generate_answer(context, question):
    prompt = f"### Context:\n{context}\n\n### Question:\n{question}\n\n### Answer:"
    inputs = tokenizer(prompt, return_tensors="pt")
    outputs = model.generate(**inputs, max_length=1000)
    answer = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return answer

In [ ]:
corpus_mixtral_expanded['respuesta'] = corpus_mixtral_expanded.apply(lambda row: generate_answer(row['frase'], row['pregunta_generada']), axis=1)


In [ ]:
corpus_mixtral_expanded


In [ ]:
def gen_pregunta(frase):
  query_engine = vector_index.as_query_engine(response_mode="compact")

  response = query_engine.query(f"Dado este texto, genera tres preguntas en español:{frase} ")

  pregunta = response.response#display_response(response)

  return pregunta

In [ ]:
corpus['Preguntas_original'] = corpus['original'].apply(lambda x: gen_pregunta(x))

In [ ]:
corpus['Preguntas_manual'] = corpus['manual'].apply(lambda x: gen_pregunta(x))

In [ ]:
corpus.head()

,original,gpt,manual,Preguntas_original,Preguntas_manual
0,4.º Que el Almanak ha de estar impreso y de ve...,El Almanak debe ser impreso y estar a la venta...,4.º El calendario debe ser impreso y estar a l...,¿Cuándo debe estar impreso y a la venta el Alm...,¿Cuál es la fecha límite para que el Almanak e...
1,En este discurso dedicado al Supremo Consejo y...,En este discurso dedicado al Supremo Consejo y...,En este discurso dedicado al Supremo Consejo y...,1. ¿Qué aspectos específicos se mencionan sobr...,1. ¿Qué se discute en el discurso dirigido al ...
2,De 12½ á una del dia 16 del pasado se perdió u...,"El día 16 del mes pasado, a las 12:30 de la ta...","El día 16 del mes pasado, entre las 12:30 y la...",¿A qué hora y en qué fecha desapareció el perr...,¿Qué raza de perro se perdió el día 16 del mes...
3,5.º Que si para el dia 1.º de noviembre no est...,Si el Almanak no está a la venta para el 1 de ...,5.º Si el Calendario no está a la venta para e...,¿Qué sucede si el Almanak no está disponible p...,¿Qué sucede si el Almanak no está disponible p...
4,¡Quién le ha dicho á ese maligno que todos los...,¿Quién le ha dicho a esa persona malintenciona...,¡Quién le ha dicho a esa persona malintenciona...,¿Quién considera a los participantes del juego...,¿Quién podría haber informado a esa persona ma...


In [ ]:
corpus.to_csv('preguntas_mistral_original_manual.csv')

# OPEN AI para traducir a español las preguntas que están en inglés

In [ ]:
from huggingface_hub import notebook_login

notebook_login()

In [ ]:
%pip install llama-index-llms-openai
%pip install llama-index
%pip install llama-index-llms-openai-like

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.4/15.4 MB 41.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 75.6/75.6 kB 7.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 141.9/141.9 kB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.7/320.7 kB 14.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 38.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 77.9/77.9 kB 11.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.3/49.3 kB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 39.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.4/290.4 kB 33.8 MB/s eta 0:00:00


In [ ]:
import openai
import os

openai.api_key="your-token"

print(openai.api_key)

In [ ]:
import pandas as pd

In [ ]:
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
llm = OpenAI(model="gpt-3.5-turbo")

In [ ]:
def traducir_pregunta(frase):
  messages = [
    ChatMessage(role="user", content=f"Translate into Spanish these phrase:{frase}"),
  ]
  resp = llm.chat(messages)

  return resp.message.content

In [ ]:
corpus = pd.read_csv('questions_original.csv')

In [ ]:
corpus['traduccion'] = corpus['query'].apply(lambda x: traducir_pregunta(x))

In [ ]:
corpus

,Unnamed: 0,query,reference_contexts,reference_answer,reference_answer_by,query_by,traduccion
0,0,What is the deadline for the Almanak to be pri...,['4.º Que el Almanak ha de estar impreso y de ...,NaN,NaN,ai (gpt-3.5-turbo),¿Cuál es la fecha límite para que el Almanaque...
1,1,How much should each printed copy of the Alman...,['4.º Que el Almanak ha de estar impreso y de ...,NaN,NaN,ai (gpt-3.5-turbo),¿Cuánto debería costar cada copia impresa del ...
2,2,What is the price for a copy of the Almanak if...,['4.º Que el Almanak ha de estar impreso y de ...,NaN,NaN,ai (gpt-3.5-turbo),¿Cuál es el precio de una copia del Almanaque ...
3,3,¿Por qué se consideran importantes las Leyes f...,['En este discurso dedicado al Supremo Consejo...,NaN,NaN,ai (gpt-3.5-turbo),¿Por qué se consideran importantes las Leyes f...
4,4,¿Cuáles son los daños que se mencionan en el d...,['En este discurso dedicado al Supremo Consejo...,NaN,NaN,ai (gpt-3.5-turbo),¿Cuáles son los daños que se mencionan en el d...
...,...,...,...,...,...,...,...
295,295,"How does the lack of acorns, for example, impa...","['No es mul facil decidir la cuestion, aunque ...",NaN,NaN,ai (gpt-3.5-turbo),"¿Cómo afecta la falta de bellotas, por ejemplo..."
296,296,Discuss the importance of the issue mentioned ...,"['No es mul facil decidir la cuestion, aunque ...",NaN,NaN,ai (gpt-3.5-turbo),Discutir la importancia del tema mencionado en...
297,297,What is the main focus of the document regardi...,"['Tarifas de los haberes, cargos, gratificacio...",NaN,NaN,ai (gpt-3.5-turbo),¿Cuál es el enfoque principal del documento co...
298,298,Can you explain the purpose of the innovations...,"['Tarifas de los haberes, cargos, gratificacio...",NaN,NaN,ai (gpt-3.5-turbo),¿Puedes explicar el propósito de las innovacio...


In [ ]:
corpus.to_csv('traduccion_original.csv')